In [1]:
#this is to build the full table withTagsB, which is 60gb from the 3gb of starter dbs. will be removed later on optimization. Unnecessary if withTagsB already built
res = conn.execute('CREATE TABLE withTagsB AS SELECT l.track_id, l.word, l.count, m.artist_id, m.title, m.artist_name, t.term FROM lyrics AS l INNER JOIN metadata.songs m ON l.track_id = m.track_id  INNER JOIN artist_term.artist_term t ON t.artist_id = m.artist_id;')

NameError: name 'conn' is not defined

In [2]:
import pickle
import math
import sqlite3
class model:
  
  def __init__(self,lmbda=1):
    self.vocab = {}#dict of word:dict pairs, inner dict is of term:(count,prob) pairs
    self.terms = {}#dict of term:count pairs
    self.termxword = {}#dict of term:dict pairs, inner dict is word:p pairs
    self.toocommon = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]
    self.lmbda = lmbda
    self.nterms = 5 # number of relevant terms allowed per input song
    self.nwords = 20 #number of relevant words to be returned per input song
   #self.lyrics
    #self.Z = 0 #for div 0 errors make sure to call calcZ after training
    self.lyricstem = []
  def calcZ(self,tag):#\sum_{word type v} (c(v, tag) + lambda))
    summ = 0
    for v in self.vocab:#make sure each v is a string, and not tuple
      summ += self.c(v,tag) + self.lmbda  
    return summ
  def p(self,word,tag):#this computes p(word|tag)
  
    return (self.c(word,tag)+self.lmbda)/self.calcZ(tag)
  def c(self,word, tag):
    #gets c(word U tag)
    if word not in self.vocab:
      print("OOV")
      return 0
    if tag not in self.vocab[word]:
      #print("hehe sparse list go brr")
      self.vocab[word][tag] = 0
    return self.vocab[word][tag]
  def loadlyricstem(self, lyricstem):
    for k in lyricstem:
      self.lyricstem.append(k[0])
  def csolo(self,q):
    return self.terms[q]
  def select(self, command='SELECT p.word, p.term FROM withTagsB AS p;'):
    self.cur = self.conn.execute(command)
  def train(self):
    for h in self.cur:
      if h[0] not in self.vocab:
        terms = {h[1]:0}
        self.vocab[h[0]] = terms
      else:
        if h[1] in self.vocab[h[0]]:
          print(self.vocab[h[0]])
          #self.vocab[h[0]][h[1]][0]+=1
        else:
          print(self.vocab[h[0]])
          #self.vocab[h[0]][h[1]][0]=1
  def primeterms(self):
    termcursor = self.conn.execute('SELECT term, COUNT(*) FROM withTagsB GROUP BY term;')
    for h in termcursor:
      self.terms[h[0]] = h[1]
  def connect(self,path = '/Downloads/mxm_dataset.db'):
    self.conn = sqlite3.connect(path)
  def firstconnect(self, path = '/Downloads/mxm_dataset.db', metapath = "/Downloads/track_metadata.db", termpath = "/Downloads/artist_term.db"):
    self.conn = sqlite3.connect(path)

    attachDatabaseSQL = "ATTACH DATABASE ? AS metadata"
    dbSpec = (metapath,)
    attachDatabaseSQL2 = "ATTACH DATABASE ? AS artist_term"
    dbSpec2 = (termpath,)
    #meta = sqlite3.connect('/content/gdrive/MyDrive/track_metadata.db')
    #term = sqlite3.connect('/content/gdrive/MyDrive/artist_term.db')

    self.res = self.conn.cursor()
    self.res.execute(attachDatabaseSQL,dbSpec)
    self.res.execute(attachDatabaseSQL2,dbSpec2)

    #all 3 databases are linked here to one connection to allow correct labels
    
  def loadvocab(self, picklename = "vocab.p"):
    self.vocab = pickle.load( open( picklename, "rb" ) )
  def savevocab(self, picklename = "vocab.p"):
    pickle.dump( self.vocab, open( "vocab.p", "wb" ) )
  
  def LyricP(self,listoflyrics):#THIS TAKES A FULL BLOCK OF LYRICS AND PARSES ON WHITESPACE. THEN RETURNS TERMS WITH HIGHEST P
    applicable = []
    #listoflyrics = lyrics.split() from string to list, doesnt stem
    
    for term in self.terms:
      sum4term = 0
      for word in listoflyrics:
        sum4term+=math.log(self.p(word[0],term))
      pairr = tuple((term,sum4term))
      applicable.append(pairr)
    print(applicable)
    applicableh = sorted(applicable,key=self.getKey)
    sortie =  (sorted(applicableh, key=lambda x:x[1], reverse=True))
    return sortie[:self.nterms] 
  def getKey(self,typle):
    return typle[1]#this is to be used with LyricP. 1 is sort by sum4term to give most applicable terms
  def otherwords(self, songtags):#input: a list of terms returns list of words with highest combined scores
    #need dict of tag:dict of word:p
    relevant = {}#dictionary of word:combined p 
    for word in self.vocab:
      if word not in self.lyricstem:
        if word not in self.toocommon:
          relevant[word]=0
          for tag in songtags: 
          
            relevant[word]+=math.log(self.p(word,tag[0]))
    print(relevant)
    relevanth = sorted(relevant.items(), key = lambda x:x[1],reverse = True)
    return relevanth[:20]
    #calculate highest probability of word|tag,tag2,etc.
    #p(word|tagi)*p(word|tagn)
 
  def inputfromfile(self,filename="testlyrics.txt"):
    f=open("testlyrics.txt","r")
    print(self.LyricP(f.read())) 
  def countselect(self):
    self.countcursor = self.conn.execute('SELECT term, COUNT(*) FROM withTagsB GROUP BY term;')
    print("done select")
  def buildterms(self):
    for term in self.countcursor:
      self.terms[term[0]]=term[1]
  def saveterms(self,picklename = "terms.p"):
    pickle.dump( self.terms, open( picklename, "wb" ) )
  def loadterms(self,picklename = "terms.p"):
    self.terms = pickle.load( open( picklename, "rb" ) )

In [5]:
#AFTER FIRST USE
myModel = model(500)
myModel.loadvocab()
myModel.loadterms()
myModel.connect()
myModel.loadlyricstem(lyricstem)

OperationalError: unable to open database file

In [4]:
myModel.loadlyricstem(lyricstem)
myModel.LyricP()#GENERATES TAGS

NameError: name 'lyricstem' is not defined

In [ ]:
myModel.otherwords()#GENERATES WORDS

In [ ]:
#TRAINS FOR FIRST USE -----ONLY NEEDED WITH NO PICKLES----i dont know if this works anymore, just use the pickles
myModel = model(500)
myModel.select()
myModel.train()
myModel.savevocab()
myModel.countselect()
myModel.buildterms()
myModel.saveterms()


done select


In [ ]:
#!/usr/bin/env python
"""
Thierry Bertin-Mahieux (2011) Columbia University
tb2332@columbia.edu
This code shows how we created bag of words for the musiXmatch
dataset. I has a command line interface, but it is mostly a library
with one main function.
This is part of the Million Song Dataset project from
LabROSA (Columbia University) and The Echo Nest.
http://labrosa.ee.columbia.edu/millionsong/
Copyright 2011, Thierry Bertin-Mahieux
This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.
This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.
You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
"""

import os
import sys
try:
    from stemming.porter2 import stem
except ImportError:
    print ('You need to install the following stemming package:')
    print ('http://pypi.python.org/pypi/stemming/1.0')
    sys.exit(0)


def lyrics_to_bow(lyrics):
    """
    Main function to stem and create bag of words.
    It is what we used for the musiXmatch dataset.
    It is heavily oriented towards English lyrics, we apologize for that.
    INPUT
        lyrics as a string
    RETURN
        dictionary word -> count
        or None if something was wrong (e.g. not enough words)
    """
    # remove end of lines
    lyrics_flat = lyrics.replace('\r', '\n').replace('\n', ' ').lower()
    lyrics_flat = ' ' + lyrics_flat + ' '
    # special cases (English...)
    lyrics_flat = lyrics_flat.replace("'m ", " am ")
    lyrics_flat = lyrics_flat.replace("'re ", " are ")
    lyrics_flat = lyrics_flat.replace("'ve ", " have ")
    lyrics_flat = lyrics_flat.replace("'d ", " would ")
    lyrics_flat = lyrics_flat.replace("'ll ", " will ")
    lyrics_flat = lyrics_flat.replace(" he's ", " he is ")
    lyrics_flat = lyrics_flat.replace(" she's ", " she is ")
    lyrics_flat = lyrics_flat.replace(" it's ", " it is ")
    lyrics_flat = lyrics_flat.replace(" ain't ", " is not ")
    lyrics_flat = lyrics_flat.replace("n't ", " not ")
    lyrics_flat = lyrics_flat.replace("'s ", " ")
    # remove boring punctuation and weird signs
    punctuation = (',', "'", '"', ",", ';', ':', '.', '?', '!', '(', ')',
                   '{', '}', '/', '\\', '_', '|', '-', '@', '#', '*')
    for p in punctuation:
        lyrics_flat = lyrics_flat.replace(p, '')
    words = filter(lambda x: x.strip() != '', lyrics_flat.split(' '))
    # stem words
    words = map(lambda x: stem(x), words)
    bow = {}
    for w in words:
        if not w in bow.keys():
            bow[w] = 1
        else:
            bow[w] += 1
    # remove special words that are wrong
    fake_words = ('>', '<', 'outro~')
    bowwords = bow.keys()
    for bw in bowwords:
        if bw in fake_words:
            bow.pop(bw)
        elif bw.find(']') >= 0:
            bow.pop(bw)
        elif bw.find('[') >= 0:
            bow.pop(bw)
    # not big enough? remove instrumental ones among others
    if len(bow) <= 3:
        return None
    # done
    return bow


def die_with_usage():
    """ HELP MENU """
    print ('lyrics_to_bow.py')
    print ('   by T. Bertin-Mahieux (2011) Columbia University')
    print ('      tb2332@columbia.edu')
    print ('This code shows how we transformed lyrics into bag-of-words.')
    print ('It is mostly intended to be used as a library, but you can pass')
    print ('in lyrics and we print the resulting dictionary.')
    print ('')
    print ('USAGE:')
    print ('  ./lyrics_to_bow.py <lyrics>')
    print ('PARAMS:')
    print ('    <lyrics>  - lyrics as one string')
    sys.exit(0)


if __name__ == '__main__':

    # help menu
    if len(sys.argv) < 2:
        die_with_usage()

    # params (lyrics)
    f=open("testlyrics.txt","r")
     
    lyrics = f.read()
    for word in sys.argv[2:]:
        lyrics += ' ' + word
    lyrics = lyrics.strip()

    # make bag of words
    bow = lyrics_to_bow(lyrics)
    if bow is None:
        print ('ERROR, maybe there was not enough words to be realistic?')
        sys.exit(0)

    # print result
    try:
        from operator import itemgetter
        lyricstem =  (sorted(bow.items(), key=itemgetter(1), reverse=True))
        print(lyricstem)
    except ImportError:
        print (bow)

[('the', 15), ('oohooh', 15), ('of', 7), ('we', 6), ('ah', 5), ('and', 5), ('ahah', 4), ('from', 4), ('your', 4), ('come', 3), ('land', 3), ('will', 3), ('our', 3), ('ice', 2), ('snow', 2), ('midnight', 2), ('sun', 2), ('where', 2), ('hot', 2), ('spring', 2), ('flow', 2), ('to', 2), ('on', 2), ('sweep', 2), ('with', 2), ('thresh', 2), ('oar', 2), ('onli', 2), ('goal', 2), ('be', 2), ('western', 2), ('shore', 2), ('how', 2), ('so', 2), ('can', 2), ('all', 2), ('hammer', 1), ('god', 1), ('drive', 1), ('ship', 1), ('new', 1), ('fight', 1), ('hord', 1), ('sing', 1), ('cri', 1), ('valhalla', 1), ('i', 1), ('am', 1), ('soft', 1), ('field', 1), ('green', 1), ('whisper', 1), ('tale', 1), ('gore', 1), ('calm', 1), ('tide', 1), ('war', 1), ('are', 1), ('overlord', 1), ('now', 1), ('you', 1), ('would', 1), ('better', 1), ('stop', 1), ('rebuild', 1), ('ruin', 1), ('for', 1), ('peac', 1), ('trust', 1), ('win', 1), ('day', 1), ('despit', 1), ('lose', 1), ('ahh', 1), ('cusersthewiappdataroamingjupyte